In [1]:
import json
from backend.db.products import product_db, Product, VehicleType, Powertrain, Color
from pprint import pprint 
import traceback

# Problem with normalization of Mini MPV
def normalize(value):
    return "Mini MPV" if value == "Mini Mpv" else value

file_name = "test.json"
with open(file_name, 'r') as file:
    data = json.load(file)
    product_list = []
    for listing in data:
        try:
            product = Product(
                name=listing["heading"],
                vehicle_type=VehicleType(normalize(listing["build"]["body_type"])),
                release_year=listing["build"]["year"],
                price_in_usd=listing.get("price") or listing.get("ref_price") or listing.get("msrp"),
                is_used=listing["inventory_type"]=="used",
                powertrain_type=Powertrain["EV" if listing["build"].get("powertrain_type") == "BEV" else "ICE" if listing["build"].get("powertrain_type") == "Combustion" else "Hybrid"],
                num_seats=listing["build"]["std_seating"],
                color=listing.get("base_ext_color"),
                specs=listing["build"],
                image_url=listing["media"]["photo_links"] if listing.get("media") else [])
            
        except Exception as e:
            traceback.print_exc()
            print("Error in parsing the following listing:")
            pprint(listing)
            continue
        
        product_list.append(product)
    
    await product_db.reset()
    await product_db.add(product_list)

Traceback (most recent call last):
  File "/var/folders/st/jgttbtbx3qz7d487_bblsjvw0000gn/T/ipykernel_5218/2504413742.py", line 16, in <module>
    product = Product(
              ^^^^^^^^
  File "/Users/viktor/Library/Caches/pypoetry/virtualenvs/backend-T0czkKg2-py3.11/lib/python3.11/site-packages/pydantic/main.py", line 176, in __init__
    self.__pydantic_validator__.validate_python(data, self_instance=self)
pydantic_core._pydantic_core.ValidationError: 1 validation error for Product
color
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/string_type
Traceback (most recent call last):
  File "/var/folders/st/jgttbtbx3qz7d487_bblsjvw0000gn/T/ipykernel_5218/2504413742.py", line 16, in <module>
    product = Product(
              ^^^^^^^^
  File "/Users/viktor/Library/Caches/pypoetry/virtualenvs/backend-T0czkKg2-py3.11/lib/python3.11/site-packages/pydantic/main.py", line 176, 

Error in parsing the following listing:
{'base_int_color': 'Black',
 'build': {'body_type': 'Sedan',
           'city_mpg': 98,
           'doors': 4,
           'drivetrain': '4WD',
           'fuel_type': 'Electric',
           'highway_mpg': 94,
           'made_in': 'USA',
           'make': 'Mercedes-Benz',
           'model': 'EQE',
           'overall_height': '59.5',
           'overall_length': '196.9',
           'overall_width': '77.2',
           'powertrain_type': 'BEV',
           'std_seating': '5',
           'transmission': 'Automatic',
           'trim': 'Base',
           'vehicle_type': 'Car',
           'year': 2024},
 'data_source': 'mc',
 'dealer': {'city': 'Temecula',
            'country': 'US',
            'dealer_type': 'franchise',
            'id': 1015559,
            'latitude': '33.536684',
            'longitude': '-117.170866',
            'msa_code': '6780',
            'name': 'Mercedes-benz Of Temecula',
            'phone': '951-330-3188',
        

In [2]:
x = await product_db.size()
print(x)

2743
